In [8]:

import os
import datetime
import copy

import os
os.environ["OMP_NUM_THREADS"] = '6'

import numpy as np
from numpy.random import normal as rnorm, multinomial as rmultinomial
import scipy.stats
from scipy.special import logsumexp, loggamma
from sklearn.mixture import GaussianMixture
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge, LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [2]:
df_ooi = pd.read_csv("data/ooi-gi01sumo-sbd12-02-flortd000_655e_fffa_3b24.csv", skiprows=1, parse_dates=["UTC"])
df_ooi = df_ooi.rename(columns = {"UTC": "time", "m": "z", "microg.L-1": "mass_concentration_of_chlorophyll_a_in_sea_water", "m-1": "flubsct", })
del df_ooi["z"]
df_ooi = df_ooi.set_index("time")

In [3]:
array_obs = df_ooi["mass_concentration_of_chlorophyll_a_in_sea_water"].resample("1D").max().to_numpy()
array_datetime = df_ooi["mass_concentration_of_chlorophyll_a_in_sea_water"].resample("1D").max().index.to_numpy()
array_cross = df_ooi["flubsct"].resample("1D").max().to_numpy()

In [4]:
list_split_index = list(np.where(np.isnan(array_obs))[0])
list_split_index.append(len(array_obs))
list_split_index.insert(0, 0)
list_subseq = []
list_subseq_cross = []
list_subseq_datetime = []
for tmp_index in range(len(list_split_index) - 1):
    tmp_index_left = list_split_index[tmp_index] + 1
    tmp_index_right = list_split_index[tmp_index + 1]
    if tmp_index_right - tmp_index_left < 20:
        continue
    list_subseq.append(array_obs[tmp_index_left:tmp_index_right])
    list_subseq_datetime.append(array_datetime[tmp_index_left:tmp_index_right])
    list_subseq_cross.append(array_cross[tmp_index_left:tmp_index_right])

In [5]:
[len(i) for i in list_subseq]

[65, 106, 175, 184, 31, 31, 225, 148, 79, 134, 130, 59, 77]

In [6]:
list_p_AR = [1, 2, ]
list_p_AR_cross = [1, 2,]
max_p = max(max(list_p_AR), max(list_p_AR_cross))

list_X, list_Y, list_exog = [], [], []
for tmp_subseq, tmp_subseq_cross, tmp_subseq_datetime in zip(list_subseq, list_subseq_cross, list_subseq_datetime):
    X, Y, exog = [], [], []
    for tmp_index_test in range(max_p, len(tmp_subseq)):
        tmp_X = []
        for tmp_p in list_p_AR:
            tmp_X.append(tmp_subseq[tmp_index_test - tmp_p])
        for tmp_p in list_p_AR_cross:
            tmp_X.append(tmp_subseq_cross[tmp_index_test - tmp_p])
        X.append(tmp_X)
        Y.append([tmp_subseq[tmp_index_test]])
        
        tmp_date= tmp_subseq_datetime[tmp_index_test]
        tmp_t = tmp_subseq_datetime[tmp_index_test].toordinal() - tmp_date.replace(month = 1, day = 1).toordinal()
        tmp_exog = [np.cos(tmp_t / 365 * 2 * np.pi)]
        exog.append(tmp_exog)
        
    list_X.append(X)
    list_Y.append(Y)
    list_exog.append(exog)
    

In [7]:
thres = 2.0

list_p_AR = [1, 2, ]
list_p_AR_cross = [1, 2,]
max_p = max(max(list_p_AR), max(list_p_AR_cross))

for start_index in range(7, 30):
    
    print(start_index)
    
    X, Y = [], []
    list_index_test_set = []
    for cc, (tmp_subseq, tmp_subseq_cross, tmp_subseq_datetime) in enumerate(zip(list_subseq, list_subseq_cross, list_subseq_datetime)):
        for tmp_index_test in range(max_p, len(tmp_subseq)):
            tmp_X = []
            tmp_date = tmp_subseq_datetime[tmp_index_test]
            tmp_tsklearn.svmseq_datetime[tmp_index_test].toordinal() - tmp_date.replace(month = 1, day = 1).toordinal()
            tmp_X.append(np.cos(tmp_t / 365 * 2 * np.pi))
            for tmp_p in list_p_AR:
                tmp_X.append(tmp_subseq[tmp_index_test - tmp_p])
            for tmp_p in list_p_AR_cross:
                tmp_X.append(tmp_subseq_cross[tmp_index_test - tmp_p])
            X.append(tmp_X)
            Y.append(tmp_subseq[tmp_index_test])

            if cc >= 2 and tmp_index_test >= start_index:
                list_index_test_set.append(len(Y) - 1)
                
    y_true, y_pred = [], []
    y_pred_naive = []
    for tmp_index in list_index_test_set:
        reg = RandomForestRegressor(n_estimators=100)
        reg.fit(X[:tmp_index], Y[:tmp_index])
        tmp_y_pred = reg.predict([X[tmp_index]])[0]
        tmp_y_true = Y[tmp_index]
        tmp_y_pred_naive = Y[tmp_index - 1]
        y_true.append(tmp_y_true)
        y_pred.append(tmp_y_pred)
        y_pred_naive.append(tmp_y_pred_naive)

    tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive), \
          roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
    print(tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive)


7
ERROR! Session/line number was not unique in database. History logging moved to new session 786
0.709972618875776 0.7330392195960626 0.9359228172611987 0.927987866070614
8
0.7062161641574005 0.7370556353652442 0.9330542961201129 0.9279605507655717
9
0.7056645059824448 0.7379790270513338 0.9334993139154915 0.9281497408911082
10
0.7116308182053872 0.7402707055082842 0.9346101007772045 0.9282670932165126
11
0.7148021139986562 0.7409740773063224 0.9337907392491601 0.9283097263422588
12
0.715280832936023 0.7416091510095923 0.9333718418805566 0.9274699884474358
13
0.7129943220052368 0.7419071004843665 0.9344247132815391 0.9265014181773339
14
0.7138692965385453 0.7398118076602047 0.9330294934255331 0.9261747603331761
15
0.7103268299138166 0.7331367278062462 0.932281362093648 0.9256167328840204
16
0.7010960410325116 0.7195698492282645 0.9320696923786312 0.9244011154293744
17
0.6854359887718915 0.7080343863155206 0.9282570199848216 0.9229895719144391
18
0.6897193765776604 0.7155710477649073 0

In [13]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [14]:
thres = 2.0

list_p_AR = [1, 2, ]
list_p_AR_cross = [1, 2,]
max_p = max(max(list_p_AR), max(list_p_AR_cross))

for start_index in range(7, 30):
    
    print(start_index)
    
    X, Y = [], []
    list_index_test_set = []
    for cc, (tmp_subseq, tmp_subseq_cross, tmp_subseq_datetime) in enumerate(zip(list_subseq, list_subseq_cross, list_subseq_datetime)):
        for tmp_index_test in range(max_p, len(tmp_subseq)):
            tmp_X = []
            tmp_date = tmp_subseq_datetime[tmp_index_test]
            tmp_t = tmp_subseq_datetime[tmp_index_test].toordinal() - tmp_date.replace(month = 1, day = 1).toordinal()
            tmp_X.append(np.cos(tmp_t / 365 * 2 * np.pi))
            for tmp_p in list_p_AR:
                tmp_X.append(tmp_subseq[tmp_index_test - tmp_p])
            for tmp_p in list_p_AR_cross:
                tmp_X.append(tmp_subseq_cross[tmp_index_test - tmp_p])
            X.append(tmp_X)
            Y.append(tmp_subseq[tmp_index_test])

            if cc >= 2 and tmp_index_test >= start_index:
                list_index_test_set.append(len(Y) - 1)
                
    y_true, y_pred = [], []
    y_pred_naive = []
    for tmp_index in list_index_test_set:
        reg = LinearSVR()
        reg.fit(X[:tmp_index], Y[:tmp_index])
        tmp_y_pred = reg.predict([X[tmp_index]])[0]
        tmp_y_true = Y[tmp_index]
        tmp_y_pred_naive = Y[tmp_index - 1]
        y_true.append(tmp_y_true)
        y_pred.append(tmp_y_pred)
        y_pred_naive.append(tmp_y_pred_naive)

    tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive), \
          roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
    print(tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive)


7
0.7566745499381192 0.7330392195960626 0.9337788522944988 0.927987866070614
8
0.7569520556729534 0.7370556353652442 0.9338811877508547 0.9279605507655717
9
0.7621589929156721 0.7379790270513338 0.9351732658094146 0.9281497408911082
10
0.7618398566284171 0.7402707055082842 0.9348598822184434 0.9282670932165126
11
0.763406091255673 0.7409740773063224 0.9344586763059153 0.9283097263422588
12
0.7635231711731302 0.7416091510095923 0.9340499271686171 0.9274699884474358
13
0.7625632759849361 0.7419071004843665 0.933232622189337 0.9265014181773339
14
0.7658295316888861 0.7398118076602047 0.9333700036670333 0.9261747603331761
15
0.7681195674582653 0.7331367278062462 0.9322491917193456 0.9256167328840204
16
0.761087931594532 0.7195698492282645 0.9313176558417321 0.9244011154293744
17
0.7513486516039644 0.7080343863155206 0.9301739888129972 0.9229895719144391
18
0.7547129983196915 0.7155710477649073 0.9290395402139987 0.9215962360204093
19
0.7516109914713363 0.7128183914569592 0.9279218671623735

In [10]:
thres = 2.0

list_p_AR = [1, 2, ]
list_p_AR_cross = [1, 2,]
max_p = max(max(list_p_AR), max(list_p_AR_cross))

for start_index in range(7, 30):
    
    print(start_index)
    
    X, Y = [], []
    list_index_test_set = []
    for cc, (tmp_subseq, tmp_subseq_cross, tmp_subseq_datetime) in enumerate(zip(list_subseq, list_subseq_cross, list_subseq_datetime)):
        for tmp_index_test in range(max_p, len(tmp_subseq)):
            tmp_X = []
            tmp_date = tmp_subseq_datetime[tmp_index_test]
            tmp_t = tmp_subseq_datetime[tmp_index_test].toordinal() - tmp_date.replace(month = 1, day = 1).toordinal()
            tmp_X.append(np.cos(tmp_t / 365 * 2 * np.pi))
            for tmp_p in list_p_AR:
                tmp_X.append(tmp_subseq[tmp_index_test - tmp_p])
            for tmp_p in list_p_AR_cross:
                tmp_X.append(tmp_subseq_cross[tmp_index_test - tmp_p])
            X.append(tmp_X)
            Y.append(tmp_subseq[tmp_index_test])

            if cc >= 2 and tmp_index_test >= start_index:
                list_index_test_set.append(len(Y) - 1)
                
    y_true, y_pred = [], []
    y_pred_naive = []
    for tmp_index in list_index_test_set:
        reg = SVR()
        reg.fit(X[:tmp_index], Y[:tmp_index])
        tmp_y_pred = reg.predict([X[tmp_index]])[0]
        tmp_y_true = Y[tmp_index]
        tmp_y_pred_naive = Y[tmp_index - 1]
        y_true.append(tmp_y_true)
        y_pred.append(tmp_y_pred)
        y_pred_naive.append(tmp_y_pred_naive)

    tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive = r2_score(y_true = y_true, y_pred = y_pred), r2_score(y_true = y_true, y_pred = y_pred_naive), \
          roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred), roc_auc_score(y_true = np.array(y_true) > thres, y_score=y_pred_naive)
    print(tmp_r2_ar, tmp_r2_naive, tmp_rocauc_ar, tmp_rocauc_naive)


7
0.6725053555197993 0.7330392195960626 0.9381123985037861 0.927987866070614
8
0.6735258980176992 0.7370556353652442 0.9380621004905605 0.9279605507655717
9
0.6729266684061255 0.7379790270513338 0.9384174317334125 0.9281497408911082
10
0.6723428883324928 0.7402707055082842 0.9380445955942396 0.9282670932165126
11
0.6738378751229666 0.7409740773063224 0.9380930397029645 0.9283097263422588
12
0.6744526468020215 0.7416091510095923 0.9375910392284896 0.9274699884474358
13
0.6753327953869876 0.7419071004843665 0.9368294487606362 0.9265014181773339
14
0.6780416246760493 0.7398118076602047 0.9368117764157368 0.9261747603331761
15
0.6759056475517351 0.7331367278062462 0.935825464995952 0.9256167328840204
16
0.6933220610499775 0.7195698492282645 0.934825329907999 0.9244011154293744
17
0.7076851242392795 0.7080343863155206 0.9337268459960086 0.9229895719144391
18
0.7038030436533969 0.7155710477649073 0.9324890869930778 0.9215962360204093
19
0.7110949059832815 0.7128183914569592 0.931202974240948